In [1]:
from socioFetcher import Downloader, GeoDataFrame, Config
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoData
import geopandas

In [7]:
dataset = ["BEA", "ACS"]
fipsList = ["26161", "26163"]
downloader = Downloader(dataset, fipsList)
downloader.download()

Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI
Loading Data for Wayne,MI
Loading Data for Washtenaw,MI


In [9]:
downloader.data["26161"]["BEA"].DataFrame

,"Washtenaw,MI_avgIncome"
1969,4708.0
1970,4678.0
1971,5131.0
1972,5590.0
1973,6154.0
1974,6389.0
1975,7194.0
1976,8027.0
1977,8927.0
1978,10071.0


In [11]:
counties = geopandas.read_file("https://www2.census.gov/geo/tiger/TIGER2016/COUNTY/tl_2016_us_county.zip")

In [2]:
counties = geopandas.read_file("https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/State_County/MapServer/1")

DriverError: '/vsimem/dbb361b807a246dea6a559e31067065e' not recognized as a supported file format.

In [15]:
counties[counties.GEOID == "26161"]

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
1333,26,161,01623021,26161,Washtenaw,Washtenaw County,06,H1,G4020,220,11460,None,A,1828428383,42628874,+42.2523272,-083.8446336,"POLYGON ((-84.130771 42.250035, -84.130912 42...."


In [ ]:
county_data = GeoData(geo_dataframe = counties,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

In [ ]:
m = Map(
    layers=(basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"), ),
    center=(52.204793, 360.121558),
    zoom=4
)

In [ ]:
m.add_layer(county_data)
m